In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import math

PATH_data = "data" # chemin vers les data pour avions
PATH_result = "../results" # chemin vers résultat du dataframe

# Avions

In [2]:
def f_to_dec(x): # spécifique à ce jeu de données
    if x[0] == "-":
        return - min_decimal(x[1:].split())
    elif (x[-3:] == "Sud" or x[-5:] == "Ouest"):
        return - min_decimal(x.split()[:3])
    else:
        return min_decimal(x.split())

# conversion degrés, minutes, secondes => décimal
def min_decimal(x):
    return round(int(x[0][:-1]) + int(x[1][:-1])/60 + int(x[2])/3600, 5)

def ecart(lat_depart,lat_arrivee,long_depart,long_arrivee): #formule de haversine
    r = 6378
    lat_depart, long_depart = math.radians(lat_depart), math.radians(long_depart)
    lat_arrivee, long_arrivee = math.radians(lat_arrivee), math.radians(long_arrivee)
    return 2*r*math.asin(math.sqrt(math.sin((lat_depart - lat_arrivee)/2)**2 + math.cos(lat_depart)*
                                   math.cos(lat_arrivee)*math.sin((long_depart-long_arrivee)/2)**2))

***Aéroports***

In [3]:
aeroports = pd.read_excel(f"{PATH_data}/aeroports.xls", header=3).drop([90, 91]).drop(axis=1, columns=["Code OACI", 
                                                                                                 "Région"])
aeroports.loc[97][["N° du département", "Libellé département"]] = [988, "Nouvelle-Calédonie"]
aeroports.loc[94][["N° du département", "Libellé département"]] = [987, "Polynésie française"]
aeroports.loc[98, aeroports.columns]= ["DZAOUDZI-PAMANDZI", "DZA", "12° 48' 17 Sud", "45° 16' 52 Est",\
                                        976, "Mayotte", "Pamandzi", "DZAOUDZI 97615"] # ajout car dans vols
aeroports = aeroports.reset_index().drop(axis=1, columns="index")
decalage = np.zeros((len(aeroports), 2)).astype(int)
ind = aeroports.loc[aeroports["N° du département"] > 100].index
decalage[ind, 0], decalage[ind, 1] = [-6, -6, -12, 2, -5, 9, 1], [-5, -5, -11, 3, -4, 10, 2] # ete,hiver
aeroports["decalage ete"], aeroports["decalage hiver"] = [decalage[:, 0], decalage[:, 1]]
aeroports = aeroports.rename(columns={"Latitude (°,',\")": "Latitude", "Longitude (°,',\")": "Longitude"})
aeroports[["Latitude", "Longitude"]] = aeroports[["Latitude", "Longitude"]].applymap(f_to_dec)
aeroports = aeroports.drop(index=ind)

***DataFrame : basique***

In [4]:
# changer dans le csv les caractères qui ne sont pas encodés en utf-8
# duplicates gardés pour avoir une meilleure moyenne des temps de trajet
vols_ete = pd.read_csv(f"{PATH_data}/flights_summer19.csv", sep=";", usecols=[5, 6, 7, 8, 9, 10], \
                        names=["depart", "arrivee", "j_depart", "j_arrivee", "h_depart", "h_arrivee"], header=0)
vols_hiver = pd.read_csv(f"{PATH_data}/flights_winter19.csv", sep=";", usecols=[5, 6, 7, 8, 9, 10], \
                          names=["depart", "arrivee", "j_depart", "j_arrivee", "h_depart", "h_arrivee"], header=0)
vols_ete["type d'heure"] = "ete"
vols_hiver["type d'heure"] = "hiver"
vols = pd.concat([vols_ete, vols_hiver])
vols = vols.loc[vols[["depart", "arrivee"]].applymap(lambda x: x in aeroports["Code IATA"].values)\
               .apply(lambda x: x[0] and x[1], axis=1)].reset_index().drop(axis=1, columns="index")
vols["depart_arrivee"] = vols[["depart", "arrivee"]].values.sum(axis=1)
routes = vols["depart_arrivee"].unique()
n_routes = len(routes)
duree, depart, arrivee = np.zeros(n_routes), [], []
for i in range(n_routes):
    array_temp = vols[["j_depart", "j_arrivee", "h_depart", "h_arrivee", "type d'heure"]]\
                     .loc[vols["depart_arrivee"] == routes[i]].values
    decalage_ete = int(aeroports["decalage ete"].loc[aeroports["Code IATA"] == routes[i][:3]].values - \
                       aeroports["decalage ete"].loc[aeroports["Code IATA"] == routes[i][3:]].values)
    decalage_hiver = int(aeroports["decalage hiver"].loc[aeroports["Code IATA"] == routes[i][:3]].values - \
                         aeroports["decalage hiver"].loc[aeroports["Code IATA"] == routes[i][3:]].values)
    for array in array_temp:
        if array[0] > array[1]:
            array[1] += 7
        d = datetime.strptime(str(array[1])+" "+array[3], "%d %H:%M")\
            -datetime.strptime(str(array[0])+" "+array[2], "%d %H:%M")
        if array[4] == "ete":
            duree[i] += d.seconds/60 + d.days*24*60 + decalage_ete*60
        else:
            duree[i] += d.seconds/60 + d.days*24*60 + decalage_hiver*60
    duree[i] = duree[i]/len(array_temp)
    depart.append(routes[i][:3])
    arrivee.append(routes[i][3:])

vols = pd.DataFrame(depart, columns=["depart IATA"])
vols["arrivee IATA"] = arrivee

IATA_Aeroport = dict(zip(aeroports["Code IATA"].values, aeroports["Nom aéroport"].values))
IATA_lat = dict(zip(aeroports["Code IATA"].values, aeroports["Latitude"].values))
IATA_long = dict(zip(aeroports["Code IATA"].values, aeroports["Longitude"].values))
dist = np.zeros(n_routes)
for i in range(n_routes):
    IATA = vols[["depart IATA", "arrivee IATA"]].iloc[i].values
    dist[i] = round(ecart(IATA_lat[IATA[0]], IATA_lat[IATA[1]], IATA_long[IATA[0]], IATA_long[IATA[1]]), 3)
vols[["depart aeroport", "arrivee aeroport"]] = vols[["depart IATA", "arrivee IATA"]].replace(IATA_Aeroport)
vols[["depart latitude", "arrivee latitude"]] = vols[["depart IATA", "arrivee IATA"]].replace(IATA_lat)
vols[["depart longitude", "arrivee longitude"]] = vols[["depart IATA", "arrivee IATA"]].replace(IATA_long)
vols["duree (min)"] = duree
vols["distance (km)"] = dist

# si il y a A->B et B->A on enlève B->A et on garde la moyenne des valeurs de durée
to_keep = []
duree = []
for v1, v2 in vols[["depart IATA", "arrivee IATA"]].values:
    if ([v1, v2] and [v2, v1]) not in to_keep:
        to_keep.append([v1, v2])
        duree.append(round(vols["duree (min)"].loc[vols[["depart IATA", "arrivee IATA"]].apply(lambda x: 
                                                (list(x) == [v1, v2]) | (list(x) == [v2, v1]), axis=1)].mean(), 2))
vols = vols.loc[vols[["depart IATA", "arrivee IATA"]].apply(lambda x: list(x) in to_keep, axis=1)]
vols["duree (min)"] = duree

col = [["point_1", "point_1", "point_1", "point_2", "point_2", "point_2", "duree (min)", "distance (km)"], 
       ["nom", "latitude", "longitude", "nom", "latitude", "longitude", "", ""]]
col = pd.MultiIndex.from_arrays(col)
df_2_vols = pd.DataFrame(columns=col)
df_2_vols[[("point_1", "nom"), ("point_1", "latitude"), ("point_1", "longitude")]] = \
                                                vols[["depart aeroport", "depart latitude", "depart longitude"]]
df_2_vols[[("point_2", "nom"), ("point_2", "latitude"), ("point_2", "longitude")]] = \
                                                vols[["arrivee aeroport", "arrivee latitude", "arrivee longitude"]]
df_2_vols[[("duree (min)", ""), ("distance (km)", "")]] = vols[["duree (min)", "distance (km)"]]
df_2_vols[("empreinte carbone (gCO2)", "")] = df_2_vols[("distance (km)", "")].apply(lambda x: round(241.5*x, 1))
df_2_vols[("prix (euros)", "")] = df_2_vols[("distance (km)", "")].apply(lambda x: round(12*x/100, 2)) # bfmtv

# créer un csv
df_2_vols.to_csv(f"{PATH_result}/df_avions.csv", index=False)

# Résultats

In [5]:
df_2_vols

point_1                                      point_2  \
                         nom  latitude longitude                      nom   
0    PARIS-CHARLES-DE-GAULLE  49.00972   2.55000        NANTES-ATLANTIQUE   
2                 PARIS-ORLY  48.71972   2.37972       MARSEILLE-PROVENCE   
4                 PARIS-ORLY  48.71972   2.37972         TOULOUSE-BLAGNAC   
6           NICE-COTE-D'AZUR  43.67000   7.20972               PARIS-ORLY   
8                 PARIS-ORLY  48.71972   2.37972        BORDEAUX-MERIGNAC   
..                       ...       ...       ...                      ...   
298        RENNES-ST-JACQUES  48.07000  -1.72972      STRASBOURG-ENTZHEIM   
300            LILLE-LESQUIN  50.56000   3.08972             PAU-PYRENEES   
302      METZ-NANCY-LORRAINE  48.97972   6.25000             PAU-PYRENEES   
304      STRASBOURG-ENTZHEIM  48.53972   7.62972             PAU-PYRENEES   
306      STRASBOURG-ENTZHEIM  48.53972   7.62972  BIARRITZ-BAYONNE-ANGLET   

                        duree (min) distance (km) empreinte carbone (gCO2)  \
     latitude longitude                                                      
0    47.15972  -1.61000       65.83       371.545                  89728.1   
2    43.43972   5.20972       80.43       626.944                 151407.0   
4    43.63000   1.37000       78.12       571.879                 138108.8   
6    48.71972   2.37972       87.49       673.845                 162733.6   
8    44.82972  -0.71972       71.06       493.196                 119106.8   
..        ...       ...         ...           ...                      ...   
298  48.53972   7.62972       77.50       694.546                 167732.9   
300  43.38000  -0.42000       95.00       842.296                 203414.5   
302  43.38000  -0.42000      100.00       807.341                 194972.9   
304  43.38000  -0.42000       95.00       846.454                 204418.6   
306  43.46972  -1.53000      100.00       904.537                 218445.7   

    prix (euros)  
                  
0          44.59  
2          75.23  
4          68.63  
6          80.86  
8          59.18  
..           ...  
298        83.35  
300       101.08  
302        96.88  
304       101.57  
306       108.54  

[154 rows x 10 columns]